
# Sugarscape Model — Mesa 2.2 (Teaching Version)

This notebook demonstrates the classic **Sugarscape model** using the Mesa 2.x framework.

It shows:
- How to define agents and an environment (grid with sugar patches)
- How to use the **RandomActivation** scheduler
- How to visualize an agent-based model with **CanvasGrid** inside Jupyter
- How to plot collected data after the simulation



> Before running, make sure you have Mesa 2.2 installed:
> ```bash
> pip install "mesa>=2.2,<3.0" matplotlib
> ```


In [1]:

from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
import matplotlib.pyplot as plt
import numpy as np
import random


## Step 1 — Define the Environment

In [2]:

class SugarPatch(Agent):
    """A patch of sugar that regenerates over time."""
    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.max_sugar = max_sugar
        self.amount = max_sugar

    def step(self):
        if self.amount < self.max_sugar:
            self.amount += 1


## Step 2 — Define the Agents

In [3]:

class SugarAgent(Agent):
    """An agent that moves toward sugar, eats it, and expends metabolism."""
    def __init__(self, unique_id, model, pos, vision, metabolism, sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.vision = vision
        self.metabolism = metabolism
        self.sugar = sugar

    def move(self):
        """Move to the cell with the most sugar within vision range."""
        best_cell = self.pos
        max_sugar = -1
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            for dist in range(1, self.vision + 1):
                x = self.pos[0] + dx * dist
                y = self.pos[1] + dy * dist
                if 0 <= x < self.model.grid.width and 0 <= y < self.model.grid.height:
                    sugar_here = 0
                    for obj in self.model.grid.get_cell_list_contents((x, y)):
                        if isinstance(obj, SugarPatch):
                            sugar_here = obj.amount
                    if sugar_here > max_sugar:
                        best_cell = (x, y)
                        max_sugar = sugar_here
        self.model.grid.move_agent(self, best_cell)

    def eat(self):
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for obj in cell_contents:
            if isinstance(obj, SugarPatch):
                self.sugar += obj.amount
                obj.amount = 0

    def step(self):
        self.move()
        self.eat()
        self.sugar -= self.metabolism
        if self.sugar <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)


## Step 3 — Define the Model

In [4]:

class SugarscapeModel(Model):
    """Sugarscape model (Mesa 2.2)."""
    def __init__(self, width=20, height=20, N=50):
        self.num_agents = N
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)

        # Create sugar patches
        for cell in self.grid.coord_iter():
            if len(cell) == 2:
                contents, (x, y) = cell
            else:
                contents, x, y = cell

            max_sugar = 1
            if (x - width/4)**2 + (y - height/4)**2 < 25 or (x - 3*width/4)**2 + (y - 3*height/4)**2 < 25:
                max_sugar = 4
            patch = SugarPatch((x, y), self, (x, y), max_sugar)
            self.grid.place_agent(patch, (x, y))
            self.schedule.add(patch)

            # Create agents
        for i in range(self.num_agents):
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            vision = random.randint(1, 6)
            metabolism = random.randint(1, 4)
            sugar = random.randint(5, 25)
            agent = SugarAgent(i, self, (x, y), vision, metabolism, sugar)
            self.grid.place_agent(agent, (x, y))
            self.schedule.add(agent)

        self.datacollector = DataCollector(
            agent_reporters={"Sugar": "sugar"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


## Step 4 — Visualization in Jupyter

In [5]:

def agent_portrayal(agent):
    if isinstance(agent, SugarPatch):
        color_intensity = int(255 * agent.amount / agent.max_sugar)
        color = f"#ffff{color_intensity:02x}"
        return {
            "Shape": "rect",
            "Filled": "true",
            "Layer": 0,
            "Color": color,
            "w": 1,
            "h": 1
        }
    elif isinstance(agent, SugarAgent):
        return {
            "Shape": "circle",
            "Color": "blue",
            "Filled": "true",
            "Layer": 1,
            "r": 0.5
        }

grid = CanvasGrid(agent_portrayal, 20, 20, 500, 500)
m = SugarscapeModel(width=20, height=20, N=50)




/Users/mhlees/Library/CloudStorage/GoogleDrive-m.h.lees@computationalscience.nl/My Drive/Teaching/2025/Model Based Decisions/Code/.venv/lib/python3.13/site-packages/mesa/agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)
/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:19: UserWarning: Agent (0, 0) is being placed with
place_agent() despite already having the position (0, 0). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(patch, (x, y))
/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:19: UserWarning: Agent (0, 1) is being placed with
place_agent() despite already having the position (0, 1). In most
cases, you'd want to clear the current position with remove_agent()
before placing the 

In [ ]:
server = ModularServer(
    SugarscapeModel,
    [grid],
    "Sugarscape Model",
    {"width": 20, "height": 20, "N": 50}
)
server.port = 8521
server.launch()


Interface starting at http://127.0.0.1:8521


/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:19: UserWarning: Agent (0, 0) is being placed with
place_agent() despite already having the position (0, 0). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(patch, (x, y))
/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:19: UserWarning: Agent (0, 1) is being placed with
place_agent() despite already having the position (0, 1). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(patch, (x, y))
/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:19: UserWarning: Agent (0, 2) is being placed with
place_agent() despite already having the position (0, 2). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(patch, (x,

RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}


/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:30: UserWarning: Agent 0 is being placed with
place_agent() despite already having the position (1, 7). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:30: UserWarning: Agent 1 is being placed with
place_agent() despite already having the position (18, 6). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:30: UserWarning: Agent 2 is being placed with
place_agent() despite already having the position (14, 4). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
/var/folders/hw/86l889_j4zq_8nlt1zg5j6gc0000gn/T/ipykernel_31278/3111202984.py:30: UserWarning: Agent 3 is being placed with


{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step","step":33}
{"type":"get_step",

## Step 5 — Post-Simulation Analysis

In [ ]:

# Run simulation (headless) for analysis
model = SugarscapeModel(width=20, height=20, N=60)
for i in range(50):
    model.step()

agent_data = model.datacollector.get_agent_vars_dataframe()
final_sugar = agent_data.xs(49, level="Step")["Sugar"]

plt.figure(figsize=(6,4))
plt.hist(final_sugar, bins=12, color="skyblue", edgecolor="black")
plt.xlabel("Sugar (Wealth)")
plt.ylabel("Number of Agents")
plt.title("Wealth Distribution after 50 Steps")
plt.tight_layout()
plt.show()



### ✅ Teaching Ideas
- Have students vary parameters (`N`, `vision`, `metabolism`) and observe changes.
- Introduce a second resource ("spice") and explore trade.
- Compute the Gini coefficient to measure inequality.
